Course Human-Centered Data Science ([HCDS](https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_human_centered_data_science.html)) - Winter Term 2020/21 - [HCC](https://www.mi.fu-berlin.de/en/inf/groups/hcc/index.html) | [Freie Universität Berlin](https://www.fu-berlin.de/)

***

# A2 - Wikipedia, ORES, and Bias in Data
Please follow the reproducability workflow as practiced during the last exercise.

## Step 1⃣ | Data acquisition

You will use two data sources: (1) Wikipedia articles of politicians and (2) world population data.

**Wikipedia articles -**
The Wikipedia articles can be found on [Figshare](https://figshare.com/articles/Untitled_Item/5513449). It contains politiciaans by country from the English-language wikipedia. Please read through the documentation for this repository, then download and unzip it to extract the data file, which is called `page_data.csv`.

**Population data -**
The population data is available in `CSV` format in the `_data` folder. The file is named `export_2019.csv`. This dataset is drawn from the [world population datasheet](https://www.prb.org/international/indicator/population/table/) published by the Population Reference Bureau (downloaded 2020-11-13 10:14 AM). I have edited the dataset to make it easier to use in this assignment. The population per country is given in millions!

In [15]:
import requests 
import zipfile
import shutil
import os
import pandas as pd
import numpy as np
import json

In [16]:
def download_zip_file(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)
            
def extract_zip_file(path):
    with zipfile.ZipFile(path, "r") as zip_ref:
        zip_ref.extractall(path.replace('.zip', ''))
    
def move_to_data_raw(dir_path, filename):
    shutil.copyfile(f"{dir_path}{filename}", f"../data_raw/{filename}")
    
def remove_zip_files(path):
    os.remove(path)
    shutil.rmtree(path.replace('.zip', ''))

def save_data_raw(data, filename):
    __save(data, '../data_raw/', filename)
    
def save_data_clean(data, filename):
    __save(data, '../data_clean/', filename)
    
def save_results(data, filename):
    __save(data, '../results/', filename)
    
def __save(data, path, filename):
    data.to_csv(f'{path}{filename}', index=False)
    
def read_data_raw(filename, sep=','):
    return __read('../data_raw/', filename, sep)
    
def read_data_clean(filename, sep=','):
    return __read('../data_clean/', filename, sep)

def read_results(filename, sep=','):
    return __read('../results/', filename, sep)

def __read(path, filename, sep=','):
    return pd.read_csv(f'{path}{filename}', sep=sep)

In [9]:
url = 'https://ndownloader.figshare.com/files/9614893'
zip_path = '_data/country.zip'

download_zip_file(url, zip_path)
extract_zip_file(zip_path)
move_to_data_raw('_data/country/country/data/', 'page_data.csv')
move_to_data_raw('_data/', 'export_2019.csv')
remove_zip_files(zip_path)

## Step 2⃣ | Data processing and cleaning
The data in `page_data.csv` contain some rows that you will need to filter out. It contains some page names that start with the string `"Template:"`. These pages are not Wikipedia articles, and should not be included in your analysis. The data in `export_2019.csv` does not need any cleaning.

***

| | `page_data.csv` | | |
|-|------|---------|--------|
| | **page** | **country** | **rev_id** |
|0|	Template:ZambiaProvincialMinisters | Zambia | 235107991 |
|1|	Bir I of Kanem | Chad | 355319463 |

***

| | `export_2019.csv` | | |
|-|------|---------|--------|
| | **country** | **population** | **region** |
|0|	Algeria | 44.357 | AFRICA |
|1|	Egypt | 100.803 | 355319463 |

***

In [11]:
page_data = read_data_raw('page_data.csv')
page_data = page_data[~page_data['page'].str.contains('Template:')] 

In [12]:
country_data = read_data_raw('export_2019.csv', ';')

### Getting article quality predictions with ORES

Now you need to get the predicted quality scores for each article in the Wikipedia dataset. We're using a machine learning system called [**ORES**](https://www.mediawiki.org/wiki/ORES) ("Objective Revision Evaluation Service"). ORES estimates the quality of an article (at a particular point in time), and assigns a series of probabilities that the article is in one of the six quality categories. The options are, from best to worst:

| ID | Quality Category |  Explanation |
|----|------------------|----------|
| 1 | FA    | Featured article |
| 2 | GA    | Good article |
| 3 | B     | B-class article |
| 4 | C     | C-class article |
| 5 | Start | Start-class article |
| 6 | Stub  | Stub-class article |

For context, these quality classes are a sub-set of quality assessment categories developed by Wikipedia editors. If you're curious, you can [read more](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment#Grades) about what these assessment classes mean on English Wikipedia. For this assignment, you only need to know that these categories exist, and that ORES will assign one of these six categories to any `rev_id`. You need to extract all `rev_id`s in the `page_data.csv` file and use the ORES API to get the predicted quality score for that specific article revision.

### ORES REST API endpoint

The [ORES REST API](https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model) is configured fairly similarly to the pageviews API we used for the last assignment. It expects the following parameters:

* **project** --> `enwiki`
* **revid** --> e.g. `235107991` or multiple ids e.g.: `235107991|355319463` (batch)
* **model** --> `wp10` - The name of a model to use when scoring.

**❗Note on batch processing:** Please read the documentation about [API usage](https://www.mediawiki.org/wiki/ORES#API_usage) if you want to query a large number of revisions (batches). 

You will notice that ORES returns a prediction value that contains the name of one category (e.g. `Start`), as well as probability values for each of the six quality categories. For this assignment, you only need to capture and use the value for prediction.

**❗Note:** It's possible that you will be unable to get a score for a particular article. If that happens, make sure to maintain a log of articles for which you were not able to retrieve an ORES score. This log should be saved as a separate file named `ORES_no_scores.csv` and should include the `page`, `country`, and `rev_id` (just as in `page_data.csv`).

You can use the following **sample code for API calls**:

In [32]:
# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/marisanest',
    'From': 'marisa.f.nest@fu-berlin.de'
}

def get_ores_data(rev_ids, headers, batch=True):
    # Define the endpoint
    # https://ores.wikimedia.org/scores/enwiki/?models=wp10&revids=807420979|807422778
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    # Convert the rev ids to a valid parameter value
    if batch:
        rev_ids = '|'.join(str(rev_id) for rev_id in rev_ids)
    else:
        rev_ids = str(rev_id)
    
    params = {
        'project' : 'enwiki',
        'model'   : 'wp10',
        'revids'  : rev_ids
    }

    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    
    return response

def get_batches(data, batch_size=50):
    return data.groupby(np.arange(len(data)) // batch_size)

def get_prediction_data(page_data):
    
    # Init the prediction data
    prediction_data = pd.DataFrame(
        {
            'page': [], 
            'country': [], 
            'rev_id': [], 
            'prediction': []
        }
    )
    
    # Init the error data
    error_data = pd.DataFrame(
        {
            'page': [], 
            'country': [], 
            'rev_id': []
        }
    )
    
    # Get page data as batches with batch_size = 50
    batches = get_batches(page_data)

    # Iterate overall all batches
    for index, batch in (batches):
        print(f'Batch {index + 1}/{len(batches)}', end='\r')
        
        # Extract all batch rev ids as list
        batch_rev_ids = batch.rev_id.values
        # Convert rev id list to 
        
        # Get ores data for all batch rev ids
        ores_data = get_ores_data(batch_rev_ids, headers)
    
        # Iterate overall all pages within the batch
        for index, page in batch.iterrows():
            try:
                # Extract prediction 
                prediction = ores_data['enwiki']['scores'][str(page.rev_id)]['wp10']['score']['prediction']
                
                # If no error occured (KeyError due to missing prediction), append prediction to prediction data
                prediction_data = prediction_data.append(
                    {
                        **dict(page), 
                        **{'prediction': prediction}
                    }, 
                    ignore_index=True
                ) 
            except KeyError:
                # If error occured (KeyError due to missing prediction), append error to error data
                error_data = error_data.append(dict(page), ignore_index=True)

    # Convert rev id columns to int
    prediction_data.rev_id = prediction_data.rev_id.astype(int)
    error_data.rev_id = error_data.rev_id.astype(int)
    
    # Return data
    return prediction_data, error_data

Sending one request for each `rev_id` might take some time. If you want to send batches you can use `'|'.join(str(x) for x in revision_ids` to put your ids together. Please make sure to deal with [exception handling](https://www.w3schools.com/python/python_try_except.asp) of the `KeyError` exception, when extracting the `prediction` from the `JSON` response.

### Combining the datasets

Now you need to combine both dataset: (1) the wikipedia articles and its ORES quality scores and (2) the population data. Both have columns named `country`. After merging the data, you'll invariably run into entries which cannot be merged. Either the population dataset does not have an entry for the equivalent Wikipedia country, or vis versa.

Please remove any rows that do not have matching data, and output them to a `CSV` file called `countries_no_match.csv`. Consolidate the remaining data into a single `CSV` file called `politicians_by_country.csv`.

The schema for that file should look like the following table:


| article_name | country | region | revision_id | article_quality | population |
|--------------|---------|--------|-------------|-----------------|------------|
| Bir I of Kanem | Chad  | AFRICA | 807422778 | Stub | 16877000 |

In [ ]:
# Get prediction data
prediction_data, prediction_error_data = get_prediction_data(page_data)

# Save prediction error data
save_data_clean(prediction_error_data, 'ORES_no_scores.csv')

# Merge prediction and country data by country
page_country_data = pd.merge(prediction_data, country_data, on=['country'], how='outer')

# Extract prediction and country error data (all rows with at least one value = NaN)
page_country_error_data = page_country_data[page_country_data.isnull().any(axis=1)]

# Save prediction and country error data
save_data_clean(page_country_error_data, 'countries_no_match.csv')

# Extract prediction and country data (all rows without any value = NaN)
page_country_data = page_country_data[~page_country_data.isnull().any(axis=1)]

# Save prediction and country data
save_data_clean(page_country_data, 'politicians_by_country.csv')

## Step 3⃣ | Analysis

Your analysis will consist of calculating the proportion (as a percentage) of articles-per-population (we can also call it `coverage`) and high-quality articles (we can also call it `relative-quality`)for **each country** and for **each region**. By `"high quality"` arcticle we mean an article that ORES predicted as `FA` (featured article) or `GA` (good article).

**Examples:**

* if a country has a population of `10,000` people, and you found `10` articles about politicians from that country, then the percentage of `articles-per-population` would be `0.1%`.
* if a country has `10` articles about politicians, and `2` of them are `FA` or `GA` class articles, then the percentage of `high-quality-articles` would be `20%`.

### Results format

The results from this analysis are six `data tables`. Embed these tables in the Jupyter notebook. You do not need to graph or otherwise visualize the data for this assignment. The tables will show:

1. **Top 10 countries by coverage**<br>10 highest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Bottom 10 countries by coverage**<br>10 lowest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Top 10 countries by relative quality**<br>10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Bottom 10 countries by relative quality**<br>10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population
1. **Regions by relative quality**<br>Ranking of regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

**❗Hint:** You will find what country belongs to which region (e.g. `ASIA`) also in `export_2019.csv`. You need to calculate the total poulation per region. For that you could use `groupby` and also check out `apply`.

In [22]:
politician_country_data = read_data_clean('politicians_by_country.csv')

**Coverage**

In [23]:
def get_coverage_data(data, groupby, agg):
    coverage_data = data.groupby(groupby)
    coverage_data = coverage_data.agg(**agg)
    coverage_data = coverage_data.reset_index()
    coverage_data['coverage'] = coverage_data.politicians / (coverage_data.population * 1e6)
    coverage_data = coverage_data.sort_values('coverage', ascending=False)
    coverage_data = coverage_data.drop(columns=['population', 'politicians'])
    coverage_data = coverage_data.reset_index(drop=True)
    
    return coverage_data

In [24]:
country_coverage_data = get_coverage_data(
    politician_country_data, 
    ['country', 'population'], 
    {
        'politicians':('population', 'count')
    }
)
region_coverage_data = get_coverage_data(
    politician_country_data, 
    ['region'], 
    {
        'population':('population', 'sum'), 
        'politicians':('population', 'count')
    }
)

1. **Top 10 countries by coverage**<br>10 highest-ranked countries in terms of number of politician articles as a proportion of country population

In [25]:
save_results(country_coverage_data.head(10), 'country_coverage_data_top_10.csv')

In [26]:
read_results('country_coverage_data_top_10.csv')

,country,coverage
0,Tuvalu,0.005400
1,Albania,0.000161
2,New Zealand,0.000157
3,Norway,0.000122
4,Moldova,0.000120
5,Estonia,0.000111
6,Finland,0.000103
7,Sao Tome and Principe,0.000100
8,Lithuania,0.000087
9,Uruguay,0.000081


2. **Bottom 10 countries by coverage**<br>10 lowest-ranked countries in terms of number of politician articles as a proportion of country population

In [47]:
save_results(country_coverage_data.tail(10), 'country_coverage_data_bottom_10.csv')

In [48]:
read_results('country_coverage_data_bottom_10.csv')

,country,coverage
0,Montenegro,1.157556e-07
1,Martinique,9.550562e-08
2,French Guiana,9.183673e-08
3,Suriname,6.611570e-08
4,Cape Verde,6.474820e-08
5,Bahamas,5.089059e-08
6,Barbados,4.878049e-08
7,Belize,3.818616e-08
8,Djibouti,3.744939e-08
9,Guyana,2.541296e-08


5. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population

In [49]:
save_results(region_coverage_data, 'region_coverage_data.csv')

In [50]:
read_results('region_coverage_data.csv')

,region,coverage
0,EUROPE,1.866929e-08
1,AFRICA,1.779978e-08
2,LATIN AMERICA AND THE CARIBBEAN,1.223086e-08
3,OCEANIA,9.577846e-09
4,NORTHERN AMERICA,4.964167e-09
5,ASIA,3.171010e-09


**Relative quality**

In [38]:
def absolut_quality_count(series):
    counter = 0
    
    for quality in series.values:
        if quality in ['GA', 'FA']:
            counter += 1;
            
    return counter;

def get_relative_quality_data(data, groupby, agg):
    relative_quality_data = data.groupby(groupby)
    relative_quality_data = relative_quality_data.agg(**agg)
    relative_quality_data = relative_quality_data.reset_index()
    
    relative_quality_data['relative_quality'] = relative_quality_data.absolut_quality / relative_quality_data.politicians
    relative_quality_data = relative_quality_data.sort_values('relative_quality', ascending=False)
    relative_quality_data = relative_quality_data.drop(columns=['politicians', 'absolut_quality'])
    relative_quality_data = relative_quality_data.reset_index(drop=True)
    
    return relative_quality_data

In [39]:
country_relative_quality_data = get_relative_quality_data(
    politician_country_data, 
    ['country'], 
    {
        'politicians': ('prediction', 'count'), 
        'absolut_quality': ('prediction', absolut_quality_count)
    }
)

region_relative_quality_data = get_relative_quality_data(
    politician_country_data, 
    ['region'], 
    {
        'politicians': ('prediction', 'count'), 
        'absolut_quality': ('prediction', absolut_quality_count)
    }
)

3. **Top 10 countries by relative quality**<br>10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality

In [9]:
save_results(country_relative_quality_data.head(10), 'country_relative_quality_data_top_10.csv')

In [10]:
read_results('country_relative_quality_data_top_10.csv')

,country,relative_quality
0,"Korea, North",0.222222
1,Saudi Arabia,0.128205
2,Romania,0.122449
3,Central African Republic,0.121212
4,Uzbekistan,0.107143
5,Mauritania,0.104167
6,Guatemala,0.084337
7,Dominica,0.083333
8,Syria,0.078125
9,Benin,0.076923


4. **Bottom 10 countries by relative quality**<br>10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality

In [11]:
save_results(country_relative_quality_data.tail(10), 'country_relative_quality_data_bottom_10.csv')

In [12]:
read_results('country_relative_quality_data_bottom_10.csv')

,country,relative_quality
0,San Marino,0.0
1,Sao Tome and Principe,0.0
2,Bahrain,0.0
3,Guyana,0.0
4,Seychelles,0.0
5,Guadeloupe,0.0
6,Solomon Islands,0.0
7,Grenada,0.0
8,Cape Verde,0.0
9,Bahamas,0.0


6. **Regions by relative quality**<br>Ranking of regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

In [13]:
save_results(region_relative_quality_data, 'region_relative_quality_data.csv')

In [14]:
read_results('region_relative_quality_data.csv')

,region,relative_quality
0,NORTHERN AMERICA,0.054450
1,ASIA,0.027029
2,EUROPE,0.022186
3,OCEANIA,0.020147
4,AFRICA,0.017387
5,LATIN AMERICA AND THE CARIBBEAN,0.014413


***

#### Credits

This exercise is slighty adapted from the course [Human Centered Data Science (Fall 2019)](https://wiki.communitydata.science/Human_Centered_Data_Science_(Fall_2019)) of [Univeristy of Washington](https://www.washington.edu/datasciencemasters/) by [Jonathan T. Morgan](https://wiki.communitydata.science/User:Jtmorgan).

Same as the original inventors, we release the notebooks under the [Creative Commons Attribution license (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/).